In [1]:
import numpy as np
import time
from collections import Counter

import tensorflow as tf
tf.__version__

/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

'1.14.0'

## Book Import, Analysis and Clean up for Network

In [2]:
text = open('HPBook1.txt', 'r', encoding = 'utf-8')
text = text.read()

print('Some sample text from the book -->')
text[:1000]

Some sample text from the book -->


"Harry Potter and the Sorcerer's Stone \n\nCHAPTER ONE \n\nTHE BOY WHO LIVED \n\nMr. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much. They were the last people you'd expect to be involved in anything strange or mysterious, because they just didn't hold with such nonsense. \n\nMr. Dursley was the director of a firm called Grunnings, which made drills. He was a big, beefy man with hardly any neck, although he did have a very large mustache. Mrs. Dursley was thin and blonde and had nearly twice the usual amount of neck, which came in very useful as she spent so much of her time craning over garden fences, spying on the neighbors. The Dursleys had a small son called Dudley and in their opinion there was no finer boy anywhere. \n\nThe Dursleys had everything they wanted, but they also had a secret, and their greatest fear was that somebody would discover it. They didn't think they could bear it if anyone found out about 

In [3]:
total_words = len(text.split())
total_characters = len(text)
unique_words = len(set(text.split()))
unique_characters = len(set(text))
paragraphs = text.split('\n\n')

print ("Total words in book :", total_words)
print ("Total characters in book :", total_characters)
print ("Unique words in book :", unique_words)
print ("Unique characters in book :", unique_characters)
print ("Total Paragraphs :", len(paragraphs))

Total words in book : 78449
Total characters in book : 442744
Unique words in book : 11897
Unique characters in book : 80
Total Paragraphs : 3033


In [4]:
most_words = Counter()
for i in range(len(paragraphs)):
    for x in paragraphs[i]:
        most_words[x] +=1

        
#most_words.most_common()

In [5]:
vocab = sorted(set(text))
vocab_to_int = {c : i for i,c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
embeddings = np.array([vocab_to_int[i] for i in text], dtype=np.int32)

#vocab_to_int

In [6]:
print ("Examples encodings (inputs to our NN) -->>")
print (text[:120])
print (embeddings[:120])

Examples encodings (inputs to our NN) -->>
Harry Potter and the Sorcerer's Stone 

CHAPTER ONE 

THE BOY WHO LIVED 

Mr. and Mrs. Dursley, of number four, Privet D
[31 51 68 68 75  1 39 65 70 70 55 68  1 51 64 54  1 70 58 55  1 42 65 68
 53 55 68 55 68  4 69  1 42 70 65 64 55  1  0  0 26 31 24 39 43 28 41  1
 38 37 28  1  0  0 43 31 28  1 25 38 48  1 46 31 38  1 35 32 45 28 27  1
  0  0 36 68 10  1 51 64 54  1 36 68 69 10  1 27 71 68 69 62 55 75  8  1
 65 56  1 64 71 63 52 55 68  1 56 65 71 68  8  1 39 68 59 72 55 70  1 27]


## RNN with LSTM Model Build and Optimization

In [7]:
def placeholders(batch_size, step_size):
    inputs = tf.placeholder(tf.int32, shape=(batch_size, step_size), name='inputs')
    targets = tf.placeholder(tf.int32, shape=(batch_size, step_size), name='targets')
    keep_prob = tf.placeholder(tf.float32, name= 'prob')
    
    return inputs, targets, keep_prob

In [8]:
def lstms(lstm_size, layers, batch_size, keep_prob):
    
    def build_cell(lstm_size, keep_prob):
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop
    
    cell = tf.contrib.rnn.MultiRNNCell([build_cell(lstm_size, keep_prob) for _ in range(layers)])
    state = cell.zero_state(batch_size, tf.float32)
    
    return cell, state

def rnn_output(lstm_output, input_size, output_size):
    sequence = tf.concat(lstm_output, axis=1)
    x = tf.reshape(sequence, [-1, input_size])
    
    with tf.variable_scope('softmax'):
        weights = tf.Variable(tf.truncated_normal(shape=(input_size, output_size), stddev=0.1))
        bias = tf.Variable(tf.zeros(output_size))
        
    logits = tf.matmul(x, weights) + bias
    predictions = tf.nn.softmax(logits, name='predictions')                          
    
    return predictions, logits

def losses(logits, y, lstm_size, num_classes):
    labels = tf.reshape(tf.one_hot(y, num_classes), logits.get_shape())
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
    
    return loss



In [9]:
batch_size = 128
num_steps = 100        
lstm_size = 512 
num_layers = 2         
learning_rate = 0.001  
prob = 0.5 
grad_clip = 5
num_classes = len(vocab)


def generate_batches(inputs, batch_size, num_steps):
    char_batch = batch_size * num_steps
    num_batches = len(inputs)//char_batch
    
    idx = char_batch * num_batches 
    inputs = inputs[:idx]
    inputs = inputs.reshape((batch_size, -1))
    
    for i in range(0, inputs.shape[1], num_steps):
        x = inputs[:, i : i+num_steps]
        y = np.zeros_like(x)
        y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
        
        yield x, y
        
batches = generate_batches(embeddings, 10, 50)
x, y = next(batches)

print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[31 51 68 68 75  1 39 65 70 70]
 [ 1 69 66 65 61 55  1 51 52 65]
 [70 69  1 52 55 51 61 10  1  0]
 [64 65 69 55 10  1  0  0  3 48]
 [70 58 55 59 68  1 56 51 53 55]
 [56 65 75 69  1 69 64 55 55 68]
 [64 70  1 57 51 59 64 69  1 70]
 [ 1 51 64 65 70 58 55 68  1 68]
 [51 68 54  1 58 59 63  1 51 64]
 [65 62 65 57 75  8  1 31 55 68]]

y
 [[51 68 68 75  1 39 65 70 70 55]
 [69 66 65 61 55  1 51 52 65 71]
 [69  1 52 55 51 61 10  1  0  0]
 [65 69 55 10  1  0  0  3 48 55]
 [58 55 59 68  1 56 51 53 55 69]
 [65 75 69  1 69 64 55 55 68 59]
 [70  1 57 51 59 64 69  1 70 58]
 [51 64 65 70 58 55 68  1 68 55]
 [68 54  1 58 59 63  1 51 64 54]
 [62 65 57 75  8  1 31 55 68 63]]


Optimization

In [10]:
def cal_optimizer(loss, learning_rate, clipping):
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), clipping)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer


tf.reset_default_graph()
inputs, targets, keep_prob = placeholders(batch_size, num_steps)
cell, initial_state = lstms(lstm_size, num_layers, batch_size, keep_prob)
one_hot_inputs = tf.one_hot(inputs, num_classes)
outputs, state = tf.nn.dynamic_rnn(cell, one_hot_inputs, initial_state = initial_state)
final_state = state
predictions, logits = rnn_output(outputs, lstm_size, num_classes)
loss = losses(logits, targets, lstm_size, num_classes)
optimizer = cal_optimizer(loss, learning_rate, grad_clip)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:

Future majo

## Train Model

In [11]:
epochs = 100
save_every_n = 300

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    counter = 0
    for e in range(epochs):
        new_state = sess.run(initial_state)
        for x, y in generate_batches(embeddings, batch_size, num_steps):
            counter += 1
            start = time.time()
            feed_dict = {inputs: x,
                    targets: y,
                    keep_prob: prob,
                    initial_state: new_state}
            batch_loss, new_state, _ = sess.run([loss, 
                                                 final_state, 
                                                 optimizer], 
                                                 feed_dict = feed_dict)
            
            end = time.time()
            print('Epoch: {}/{}... '.format(e+1, epochs),
                  'Training Step: {}... '.format(counter),
                  'Training loss: {:.4f}... '.format(batch_loss),
                  '{:.4f} sec/batch'.format((end-start)))
        
            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

Epoch: 1/100...  Training Step: 1...  Training loss: 4.3839...  0.6318 sec/batch
Epoch: 1/100...  Training Step: 2...  Training loss: 4.3027...  0.4848 sec/batch
Epoch: 1/100...  Training Step: 3...  Training loss: 3.8550...  0.4845 sec/batch
Epoch: 1/100...  Training Step: 4...  Training loss: 5.5001...  0.4903 sec/batch
Epoch: 1/100...  Training Step: 5...  Training loss: 4.1873...  0.4768 sec/batch
Epoch: 1/100...  Training Step: 6...  Training loss: 3.8436...  0.5012 sec/batch
Epoch: 1/100...  Training Step: 7...  Training loss: 3.7455...  0.6614 sec/batch
Epoch: 1/100...  Training Step: 8...  Training loss: 3.6523...  0.5864 sec/batch
Epoch: 1/100...  Training Step: 9...  Training loss: 3.5230...  0.5633 sec/batch
Epoch: 1/100...  Training Step: 10...  Training loss: 3.4486...  0.4742 sec/batch
Epoch: 1/100...  Training Step: 11...  Training loss: 3.4354...  0.5082 sec/batch
Epoch: 1/100...  Training Step: 12...  Training loss: 3.4255...  0.5222 sec/batch
Epoch: 1/100...  Training

In [12]:
#tf.train.get_checkpoint_state('checkpoints')

In [13]:
def select_characters(predictions, vocab_size, top = 5):
    probs = np.squeeze(predictions)
    probs[np.argsort(probs)[: -top]] = 0
    probs = probs / np.sum(probs)
    chars = np.random.choice(vocab_size, 1, p=probs)[0]
    return chars

In [14]:
def sample_and_select(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    samples = [c for c in prime] 
    batch_size, num_steps = 1, 1
    num_layers = 2
    num_classes = len(vocab)
    learning_rate = 0.001  
    grad_clip = 5
    
    tf.reset_default_graph()
    inputs, targets, keep_prob = placeholders(batch_size, num_steps)
    cell, initial_state = lstms(lstm_size, num_layers, batch_size, keep_prob)
    one_hot_inputs = tf.one_hot(inputs, num_classes)
    outputs, state = tf.nn.dynamic_rnn(cell, one_hot_inputs, initial_state = initial_state)
    final_state = state
    prediction, logits = rnn_output(outputs, lstm_size, num_classes)
    loss = losses(logits, targets, lstm_size, num_classes)
    optimizer = cal_optimizer(loss, learning_rate, grad_clip)

    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(initial_state)
        
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {inputs: x,
                    keep_prob: 1.,
                    initial_state: new_state}
            preds, new_state = sess.run([prediction, final_state], 
                                         feed_dict=feed)

        c = select_characters(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {inputs: x,
                    keep_prob: 1.,
                    initial_state: new_state}
            preds, new_state = sess.run([prediction, final_state], 
                                         feed_dict=feed)

            c = select_characters(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

In [15]:
#tf.train.latest_checkpoint('checkpoints')

# Sampling New Book Text Output

In [16]:
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample_and_select(checkpoint, 2000, lstm_size, len(vocab), prime='It was a ')
print(samp)

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from checkpoints/i3400_l512.ckpt
It was a powerful hand on his face. He had been clinking at them, and their for head shot. He went raised in his face. "Then you've got the train to tell him from Gryffindor to have all about the Sorcerer's Stone, it shoused something that looked like a book full of shudly with Hagrid, though. 

He hadmed the floor in the far points from Gren fack, the books were glowing. The lips had brought his beht in fire pace to thy dirn't leak. He lat off his break, and the pull dooring plited of a scrook. 

Harry looked over his head and signer, left the cat as the theerest to his face what he was this too funly sounded them. He left to his for him. 

"Yes, think you," Harry chanked at hard. 

"I shall see you somewhere," said Dumbledore, "and well -- I don't want to get the letter, Peeves," Harry sat down at the back of Harry's shouted, but

In [17]:
checkpoint = 'checkpoints/i3400_l512.ckpt'
samp = sample_and_select(checkpoint, 2000, lstm_size, len(vocab), prime="It was")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i3400_l512.ckpt
It was the one of the storm. They couldn't melt about he had a school closer that he was glumber back to the door, but he wasn't sure they didn't speak to the plood and sat a look withor them and the signerous, and the ball back on his hand. 

"How are you got a lot here," Ron muttered, "I think it's ground," said Hermione. 

"Think yeh feel in," said Harry, trying to stup dow they left the fooest, all three of them weren't allowed to the portrait hole was only to him. Not manyed. He hadn't spell. 

"I'm getting up?" 

"Oh, you could hard leave him off the coldants," said Ron. "She hasn't heard it again. It's outta tree. He looked it again as the Stind at a somenow. I seem the table and ticket his eyes alone. 

His much face. 

"It's toucher the platers things that ded, is too?" said Harry. "Wizer alone --" 

"I don't think," said Harry, "I seel the Snate cloak in the face, Malfoy with the one pairsuof were scrating 

In [18]:
checkpoint = 'checkpoints/i3000_l512.ckpt'
samp = sample_and_select(checkpoint, 2000, lstm_size, len(vocab), prime="It was")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i3000_l512.ckpt
It was going on. 

Harry cuuldn't says out of the sprint where they speaked at them to the big orean. They hadn't seened to be still straight, showed mose tranning with his shoulder and had a bottle of crowd with a little motch forehead. He hed it around, but he couldn't have anyone seemed to great had to be so speace, to he clased to the direction. Harry had been londing to see yours, and they couldn't believe he was still so much. Hermione. A shave fly the school, the Professor McGonagall. Hagrid should have been a bin a seatury. There was a vigat madch in this corter, but the groan was around an into his hand and stepped drawing at the straight because they'd had it and get him for him. He winhed him a marble sigh fall. He was standing in the dark without enging in a cas he could. "It stand what they always have to see anything, he hadn to show you." 

"So --" 

"I know who you wait, Potter, something -- any," sai